<a href="https://colab.research.google.com/github/hayden-huynh/CSE-5334-TermProject/blob/master/CSE_5334_TermProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 5334 - Data Mining: Term Project

## About The Project
- Dataset: [Top Reddit Posts and Comments](https://www.kaggle.com/datasets/tushar5harma/topredditcomments?select=Top_Posts.csv)
- Goal: Given a comment of a post, classify which subreddit (Machine Learning, Artificial Intelligence, or Data Science) that it belongs to
- Solution: Naive Bayes Classifier for Text Classification

## Project Steps
1. Download the Top Reddit Posts and Comments [dataset](https://www.kaggle.com/datasets/tushar5harma/topredditcomments?select=Top_Posts.csv) from Kaggle
2. Based on the original dataset, formulate a dataframe consisting of a column of **comment text**, and a column of subreddit class
3. Perform text pre-processing
4. Split the data into **train**, **dev**, and **test** subsets
5. Train the Naive Bayes Classifier
6. Experiment for optimization with **dev** dataset
  - Smoothing
  - Stopword removal
  - Stemming / Lemmatization
7. Conclude final accuracy with **test** dataset

# Download the Dataset